In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
from pokemon_card_generator.data import card_data

cards_df = card_data.get_card_data()

In [3]:
cards_df.head()

,id,name,subtypes,hp,types,evolvesFrom,abilities,attacks,weaknesses,convertedRetreatCost,...,attack1_name,attack1_cost,attack1_converted_cost,attack1_damage,attack1_text,attack2_name,attack2_cost,attack2_converted_cost,attack2_damage,attack2_text
0,base1-1,Alakazam,Stage 2,80.0,Psychic,Kadabra,"[{'name': 'Damage Swap', 'text': 'As often as ...","[{'name': 'Confuse Ray', 'cost': ['Psychic', '...",Psychic,3.0,...,Confuse Ray,"[Psychic, Psychic, Psychic]",3.0,30,"Flip a coin. If heads, the Defending Pokémon i...",,,,,
1,base1-2,Blastoise,Stage 2,100.0,Water,Wartortle,"[{'name': 'Rain Dance', 'text': 'As often as y...","[{'name': 'Hydro Pump', 'cost': ['Water', 'Wat...",Lightning,3.0,...,Hydro Pump,"[Water, Water, Water]",3.0,40+,Does 40 damage plus 10 more damage for each Wa...,,,,,
2,base1-3,Chansey,Basic,120.0,Colorless,NaN,NaN,"[{'name': 'Scrunch', 'cost': ['Colorless', 'Co...",Fighting,1.0,...,Scrunch,"[Colorless, Colorless]",2.0,,"Flip a coin. If heads, prevent all damage done...",Double-edge,"[Colorless, Colorless, Colorless, Colorless]",4,80,Chansey does 80 damage to itself.
3,base1-4,Charizard,Stage 2,120.0,Fire,Charmeleon,"[{'name': 'Energy Burn', 'text': 'As often as ...","[{'name': 'Fire Spin', 'cost': ['Fire', 'Fire'...",Water,3.0,...,Fire Spin,"[Fire, Fire, Fire, Fire]",4.0,100,Discard 2 Energy cards attached to Charizard i...,,,,,
4,base1-5,Clefairy,Basic,40.0,Colorless,NaN,NaN,"[{'name': 'Sing', 'cost': ['Colorless'], 'conv...",Fighting,1.0,...,Sing,[Colorless],1.0,,"Flip a coin. If heads, the Defending Pokémon i...",Metronome,"[Colorless, Colorless, Colorless]",3,,Choose 1 of the Defending Pokémon's attacks. M...


In [ ]:
cards_df[cards_df.attacks.isna()].head()

In [ ]:
[item for item in cards_df.attacks]

In [4]:
attack3_name = [
    item[2]['name'] if isinstance(item, list) and len(item) > 2 else ''
    for item in cards_df.attacks
]

attack3_cost = [
    item[2]['cost'] if isinstance(item, list) and len(item) > 2 else ''
    for item in cards_df.attacks
]

attack3_converted_cost = [
    len(cost) if isinstance(cost, list) and len(cost) > 1 else cost
    for cost in attack3_cost
]

attack3_damage = [
    item[2]['damage'] if isinstance(item, list) and len(item) > 2 else ''
    for item in cards_df.attacks
]

attack3_text = [
    item[2]['text'] if isinstance(item, list) and len(item) > 2 else ''
    for item in cards_df.attacks
]

cards_df['attack3_name'] = attack3_name
cards_df['attack3_cost'] = attack3_cost
cards_df['attack3_converted_cost'] = attack3_converted_cost
cards_df['attack3_damage'] = attack3_damage
cards_df['attack3_text'] = attack3_text

In [16]:
cards_df[cards_df.name == 'Charizard-GX']

,id,name,subtypes,hp,types,evolvesFrom,abilities,attacks,weaknesses,convertedRetreatCost,...,attack2_name,attack2_cost,attack2_converted_cost,attack2_damage,attack2_text,attack3_name,attack3_cost,attack3_converted_cost,attack3_damage,attack3_text
9304,smp-SM60,Charizard-GX,Stage 2,250.0,Fire,Charmeleon,NaN,"[{'name': 'Wing Attack', 'cost': ['Colorless',...",Lightning,2.0,...,Crimson Storm,"[Fire, Fire, Fire, Colorless, Colorless]",5,300,Discard 3 Fire Energy from this Pokémon.,Raging Out-GX,"[Fire, Colorless, Colorless]",3,,Discard the top 10 cards of your opponent's de...
9441,smp-SM195,Charizard-GX,Stage 2,250.0,Fire,Charmeleon,NaN,"[{'name': 'Raging Destruction', 'cost': ['Fire...",Grass,4.0,...,Steam Artillery,"[Fire, Fire, Colorless, Colorless, Colorless]",5,200,,Dreadful Flames-GX,"[Fire, Fire, Colorless, Colorless, Colorless]",5,250,Discard an Energy from each of your opponent's...
9457,smp-SM211,Charizard-GX,Stage 2,250.0,Fire,Charmeleon,NaN,"[{'name': 'Flamethrower', 'cost': ['Fire', 'Fi...",Fighting,3.0,...,Flare Blitz-GX,"[Fire, Fire, Colorless, Colorless]",4,300,(You can't use more than 1 GX attack in a game.),,,,,
9691,sm3-20,Charizard-GX,Stage 2,250.0,Fire,Charmeleon,NaN,"[{'name': 'Wing Attack', 'cost': ['Colorless',...",Fighting,2.0,...,Crimson Storm,"[Fire, Fire, Fire, Colorless, Colorless]",5,300,Discard 3 Fire Energy from this Pokémon.,Raging Out-GX,"[Fire, Colorless, Colorless]",3,,Discard the top 10 cards of your opponent's de...
9828,sm3-150,Charizard-GX,Stage 2,250.0,Fire,Charmeleon,NaN,"[{'name': 'Wing Attack', 'cost': ['Colorless',...",Fairy,2.0,...,Crimson Storm,"[Fire, Fire, Fire, Colorless, Colorless]",5,300,Discard 3 Fire Energy from this Pokémon.,Raging Out-GX,"[Fire, Colorless, Colorless]",3,,Discard the top 10 cards of your opponent's de...
11599,sm115-9,Charizard-GX,Stage 2,250.0,Fire,Charmeleon,NaN,"[{'name': 'Flamethrower', 'cost': ['Fire', 'Fi...",NaN,3.0,...,Flare Blitz-GX,"[Fire, Fire, Colorless, Colorless]",4,300,(You can't use more than 1 GX attack in a game.),,,,,
11708,sma-SV49,Charizard-GX,Stage 2,250.0,Fire,Charmeleon,NaN,"[{'name': 'Wing Attack', 'cost': ['Colorless',...",NaN,2.0,...,Crimson Storm,"[Fire, Fire, Fire, Colorless, Colorless]",5,300,Discard 3 Fire Energy from this Pokémon.,Raging Out-GX,"[Fire, Colorless, Colorless]",3,,Discard the top 10 cards of your opponent's de...


In [ ]:
[isinstance(item, list)for item in cards_df.attacks]